# Лабораторная работа №5
## Выполнил студент группы Гребнев Ярослав БФИ2302

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
4. [Вывод](#Вывод)

> Дополнительные модули, использованные при выполнение лабораторной

In [ ]:
# Необходим при замере скорости выполнения кода
from datetime import datetime as t
# Нужен для создания словаря в алг. Бойера-Мура
from collections import defaultdict

### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

#### Алгоритм Кнута-Морриса-Пратта

In [ ]:
from datetime import datetime
def knut(text, pattern, ignore_case=False):
    if ignore_case:
        text, pattern = text.lower(), pattern.lower()
    prefix = [0] * len(pattern)
    for i in range(1, len(pattern)):
        j = prefix[i - 1]
        while j > 0 and pattern[i] != pattern[j]:
            j = prefix[j - 1]
        prefix[i] = j + (pattern[i] == pattern[j])
    j = 0
    for i, c in enumerate(text):
        while j > 0 and c != pattern[j]:
            j = prefix[j - 1]
        if c == pattern[j]:
            j += 1
            if j == len(pattern):
                return i - j + 1
    return -1
text = input("Строка: ")
pattern = input("Подстрока: ")
ignore_case = input("Игнорировать регистр? (y/n): ").lower() == 'y'
start = datetime.now()
pos = knut(text, pattern, ignore_case)
kmp_time = (datetime.now() - start).total_seconds()
start = datetime.now()
std_pos = text.lower().find(pattern.lower()) if ignore_case else text.find(pattern)
std_time = (datetime.now() - start).total_seconds()
print(f"\nKнут: позиция {pos}, время {kmp_time:.6f} сек")
print(f"Стандартный: позиция {std_pos}, время {std_time:.6f} сек")


Kнут: позиция -1, время 0.000119 сек
Стандартный: позиция -1, время 0.000087 сек


#### Упрощенный алгоритм Бойера-Мура

In [1]:
from datetime import datetime
from collections import defaultdict

def boyer_moore(text, pattern, ignore_case=False):
    if ignore_case:
        text, pattern = text.lower(), pattern.lower()

    d = defaultdict(lambda: len(pattern))
    for i in range(len(pattern) - 1):
        d[pattern[i]] = len(pattern) - 1 - i

    i = len(pattern) - 1
    while i < len(text):
        j = len(pattern) - 1
        while j >= 0 and text[i] == pattern[j]:
            i, j = i - 1, j - 1
        if j == -1:
            return i + 1
        i += d.get(text[i], len(pattern))
    return -1


text = input("Строка: ")
pattern = input("Подстрока: ")
ignore_case = input("Игнорировать регистр? (y/n): ").lower() == 'y'

start = datetime.now()
pos = boyer_moore(text, pattern, ignore_case)
bm_time = (datetime.now() - start).total_seconds()

start = datetime.now()
std_pos = text.lower().find(pattern.lower()) if ignore_case else text.find(pattern)
std_time = (datetime.now() - start).total_seconds()

print(f"\nБоер: позиция {pos}, время {bm_time:.6f} сек")
print(f"Стандартный: позиция {std_pos}, время {std_time:.6f} сек")


Боер: позиция 0, время 0.000076 сек
Стандартный: позиция 0, время 0.000070 сек


### Задание №2
Написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.
#### Входные данные: массив чисел, представляющий собой расстановку в
Порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.
#### Выходные данные: если решения нет, то функция должна вернуть
Пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента. 

In [ ]:
from collections import deque
def get_neighbors(state):
    neighbors = []
    size = int(len(state) ** 0.5)
    empty_pos = state.index(0)
    moves = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    row, col = divmod(empty_pos, size)
    for move in moves:
        new_row, new_col = row + move[0], col + move[1]
        if 0 <= new_row < size and 0 <= new_col < size:
            new_empty_pos = new_row * size + new_col
            new_state = list(state)
            new_state[empty_pos], new_state[new_empty_pos] = new_state[new_empty_pos], new_state[empty_pos]
            neighbors.append((new_state, new_empty_pos))

    return neighbors
def solve_puzzle(start_state):
    goal_state = tuple(range(1, len(start_state))) + (0,)
    start_state = tuple(start_state)
    if start_state == goal_state:
        return []
    queue = deque([(start_state, [], start_state.index(0))])
    visited = set()
    visited.add(start_state)
    while queue:
        state, path, empty_pos = queue.popleft()
        for neighbor_state, new_empty_pos in get_neighbors(state):
            if neighbor_state not in visited:
                visited.add(neighbor_state)
                new_path = path + [state[new_empty_pos]]
                if neighbor_state == goal_state:
                    return new_path
                queue.append((neighbor_state, new_path, new_empty_pos))
    return []
if __name__ == "__main__":
    start_state = list(map(int, input("состояние: ").split()))
    result = solve_puzzle(start_state)
    if result:
        print("победа, шаги", result)
    else:
        print("невозможно.")

### Вывод